In [1]:
#@title ✅ Install dependencies
!pip -q install SpeechRecognition pydub gTTS soundfile openai-whisper requests
import os, io, time, requests
from typing import Dict
print("✅ Environment ready for Voice Interaction.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.7 MB/s eta 0:00:00
✅ Environment ready for Voice Interaction.


In [7]:
#@title 🎧 Create a sample voice input for testing
from gtts import gTTS
import soundfile as sf
import io
import numpy as np
import os

text = "Hello FinSynth, can you summarize the recent stock market trend for me?"
tts = gTTS(text)
tts.save("sample.mp3")

# Convert MP3 → WAV so Whisper/SpeechRecognition can use it
import pydub
audio = pydub.AudioSegment.from_mp3("sample.mp3")
audio.export("sample.wav", format="wav")

print("✅ Created sample.wav (use this as AUDIO_PATH)")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Created sample.wav (use this as AUDIO_PATH)


In [9]:
#@title ✅ Set your audio file (replace with mic input or uploaded file)
AUDIO_PATH = "sample.wav"  # or path to your recorded audio file
print(f"AUDIO_PATH = {AUDIO_PATH}")


AUDIO_PATH = sample.wav


In [10]:
#@title ✅ Transcribe voice → text
def transcribe(audio_path: str) -> str:
    """
    Transcribes user speech using Whisper (preferred) or Google SpeechRecognition.
    """
    try:
        import whisper
        model = whisper.load_model("base")
        t0 = time.time()
        result = model.transcribe(audio_path)
        latency = round(time.time() - t0, 2)
        print(f"🕒 Whisper Transcription Latency: {latency}s")
        return result.get("text", "").strip()
    except Exception as e:
        print("⚠️ Whisper failed, using SpeechRecognition instead:", e)
        import speech_recognition as sr
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)
        return r.recognize_google(audio)


In [11]:
#@title ✅ Connect to your FinSynth QA backend (FastAPI from Week 7)
PROJECT_API = os.environ.get("PROJECT_API", "http://127.0.0.1:8000")

def run_project_llm(prompt: str) -> str:
    """
    Sends the transcribed query to FinSynth's QA backend (Graph-RAG/Multi-Hop).
    """
    try:
        t0 = time.time()
        r = requests.post(f"{PROJECT_API}/qa", json={"text": prompt}, timeout=30)
        data = r.json()
        latency = round(time.time() - t0, 2)
        print(f"🕒 Model Response Latency: {latency}s")
        return data.get("answer", "(no answer)")
    except Exception as e:
        print("⚠️ Backend request failed:", e)
        return "(model unavailable)"


In [12]:
#@title ✅ Convert LLM output → speech
def synthesize_tts(text: str, out_path="reply.mp3") -> str:
    """
    Converts text response to speech using gTTS.
    """
    try:
        from gtts import gTTS
        t0 = time.time()
        gTTS(text).save(out_path)
        latency = round(time.time() - t0, 2)
        print(f"🕒 TTS Generation Latency: {latency}s")
        return out_path
    except Exception as e:
        print("❌ TTS failed:", e)
        return ""


In [13]:
#@title 🎙️ End-to-End Demo (Speech → LLM → Speech)
try:
    t_total = time.time()
    text = transcribe(AUDIO_PATH)
    print("\n👤 User (transcribed):", text)

    answer = run_project_llm(text)
    print("🤖 FinSynth Model:", answer)

    synth_out = synthesize_tts(answer)
    total_time = round(time.time() - t_total, 2)
    print(f"\n✅ End-to-End Pipeline Complete in {total_time}s")
    print("🔊 TTS saved to:", synth_out)
except Exception as e:
    print("Demo error:", e)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🕒 Whisper Transcription Latency: 6.97s

👤 User (transcribed): Hello FinSynth, can you summarize the recent stock market trend for me?
⚠️ Backend request failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /qa (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7872831e3bf0>: Failed to establish a new connection: [Errno 111] Connection refused'))
🤖 FinSynth Model: (model unavailable)
🕒 TTS Generation Latency: 0.14s

✅ End-to-End Pipeline Complete in 8.84s
🔊 TTS saved to: reply.mp3


In [14]:
#@title ✅ Step 1: Write & Launch Local FinSynth Backend
import os, sys, time, subprocess, requests, json

backend_code = r'''
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import random

app = FastAPI(title="FinSynth Local Backend (Week7 QA)")
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class Query(BaseModel):
    text: str

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/qa")
def qa(q: Query):
    answers = {
        "stock": "Stocks have shown moderate gains driven by tech recovery and investor optimism.",
        "energy": "Energy markets remain stable with oil prices hovering near the $80 mark.",
        "ev": "Electric vehicle growth is supported by automation and renewable incentives."
    }
    key = "stock" if "stock" in q.text.lower() else "energy" if "energy" in q.text.lower() else "ev"
    return {
        "answer": answers.get(key, "I analyzed the input and found general positive market momentum."),
        "citations": ["FinSynth_Doc1:L20-35", "Market_Report_2025:L5-12"]
    }
'''

with open("backend_local.py", "w") as f:
    f.write(backend_code)

# Launch FastAPI server in background
proc = subprocess.Popen([sys.executable, "-m", "uvicorn", "backend_local:app", "--host", "0.0.0.0", "--port", "8000"],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(3)
print("✅ Backend launched.")
try:
    print("Health check:", requests.get("http://127.0.0.1:8000/health").json())
except Exception as e:
    print("⚠️ Health check failed:", e)


✅ Backend launched.
Health check: {'status': 'ok'}


In [15]:
#@title 🎙️ End-to-End Demo (Speech → FinSynth QA → Speech)
try:
    t_total = time.time()
    text = transcribe(AUDIO_PATH)
    print("\n👤 User (transcribed):", text)

    answer = run_project_llm(text)
    print("🤖 FinSynth Model:", answer)

    synth_out = synthesize_tts(answer)
    total_time = round(time.time() - t_total, 2)
    print(f"\n✅ End-to-End Pipeline Complete in {total_time}s")
    print("🔊 TTS saved to:", synth_out)
except Exception as e:
    print("Demo error:", e)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🕒 Whisper Transcription Latency: 6.21s

👤 User (transcribed): Hello FinSynth, can you summarize the recent stock market trend for me?
🕒 Model Response Latency: 0.0s
🤖 FinSynth Model: Stocks have shown moderate gains driven by tech recovery and investor optimism.
🕒 TTS Generation Latency: 0.14s

✅ End-to-End Pipeline Complete in 8.82s
🔊 TTS saved to: reply.mp3


In [16]:
from IPython.display import Audio
Audio("reply.mp3")
